In [1]:
import pandas as pd
import mysql.connector
from neo4j import GraphDatabase
import datetime
from datetime import timedelta

In [ ]:
#Establish connection to SQL db
df_pw = pd.read_csv("pw.csv", delimiter=",")
con = mysql.connector.connect(
    host="localhost",
    user=df_pw["name"][0], 
    passwd=df_pw["password"][0],
    database="DarknetScrape")

In [3]:
#Establish connection to Neo4j
db = GraphDatabase.driver("neo4j://127.0.0.1:7687", auth=(df_pw["name"][0], df_pw["password"][0]))
session = db.session()

In [ ]:
#Load all BM reviews
berlusconi_reviews  = pd.read_csv('reviews_price_btc.csv')
berlusconi_reviews.time = pd.to_datetime(berlusconi_reviews.time)

In [ ]:
j_addresses=pd.read_csv('candidates_final.csv')
j_addresses=j_addresses.drop(columns=['Unnamed: 0'])
j_addresses.head(1)

In [ ]:
vendor_wal_filtered=pd.read_csv('vendor_wallets_filtered.csv')

In [ ]:
query = """select a.vendor_id, b.username from (
SELECT vendor_id, count(*) as count
from vendors
group by vendor_id
) as a
JOIN (SELECT vendor_id, username from vendors where market = 'Cryptonia') b
on a.vendor_id = b.vendor_id
where a.count = 2;
"""
cursor.execute(query)
df = cursor.fetchall()
df=df[df['username'].isin(j_addresses['Vendor'])]
berlusconi_reviews = berlusconi_reviews[berlusconi_reviews["vendor_id"].isin(df["vendor_id"])]
berlusconi_reviews.head()

In [ ]:
neo_results_a = []
neo_query = session.run("""
    Match (vendor:Address) where vendor.address='3DdUb8Tt59tvZacikVt6R9pYWYqzGqgxff' 
with vendor
match (vendor)-[r:SENDS]->(t:Transaction)
with t
match (vendor2:Address)-[s:SENDS]->(t)
return distinct vendor2.address
    """)
    
for r in neo_query:
    neo_results_a.append(r[0])

In [ ]:
len(list(vendor_wal_filtered[vendor_wal_filtered['vendor_id']==i.vendor_id]['Address']))

In [ ]:

transaction_t2 = pd.DataFrame()
for i in berlusconi_reviews[berlusconi_reviews['vendor_id'].isin(vendor_wal_filtered['vendor_id'])].itertuples():
    neo_results = []
    neo_results2 = []

    end = str(pd.to_datetime(i.time.date()))
    minSum = str(round(i.price_btc_reviews_satoshi,0))
    maxSum = str(round(i.price_btc_reviews_satoshi*1.40,0))
    addresses = str(list(vendor_wal_filtered[vendor_wal_filtered['vendor_id']==i.vendor_id]['Address']))
    
    neo_query = session.run("""
    MATCH (vendor:Address)
        WHERE vendor.address IN """ + addresses + """
    WITH vendor, '""" + end + """' as endDate, """
    + minSum + """ as minSum
    MATCH (t:Transaction)-[r:RECEIVES]->(vendor)
        WHERE   ( t.outDegree = 2 OR t.outDegree = 1 ) AND t.date = date(endDate) 
        and minSum  <= r.value <= minSum * 1.03
    RETURN t, r.value
    """)
    
    for r in neo_query:
        neo_results.append(r[0])
        neo_results2.append(r[1])

    
    df_neo4j = pd.DataFrame(neo_results)
    df_neo4j["vendor"] = i.vendor_id
    df_neo4j["review"] = i.vendor_review_id
    df_neo4j["rvalue"] = pd.Series(neo_results2)
    transaction_t2 = pd.concat([transaction_t2,df_neo4j])
    
display(transaction_t2)

In [ ]:
# Assumptions:
# - inDegree: we don't know the inDegree, there might be multiple mixing addresses spending to the vendor
#           - if we want to reduce the amount of result it could be changed to e.g. <= X
# - outDegree: it is likely that only one vendor address receives the money, it might be that money is going back as change
#           - outDegree = 2: change is going back to the market
#           - outDegree = 1: exact amount is transferred to the vendor without change
#           - outDegree > 2: A) several vendor addresses of the same vendor wallet are paid at the same time 
#                            B) several vendors are paid at the same time
# (t)-[r1:RECEIVES]->(c:Address)-[s:SENDS]->(t) filter for change going back

#Backup neo_query for copy&paste
    #neo_query = session.run("""
    #MATCH (vendor:Address)
    #    WHERE vendor.address IN """ + addresses + """
    #WITH vendor, 
    #'""" + start + """' as startDate,
    #'""" + end + """' as endDate,
    #"""+ minSum + """ as minSum, 
    #""" + maxSum + """ as maxSum 
    #
    #MATCH (t:Transaction)-[r:RECEIVES]->(vendor)
    #    WHERE date(startDate) <= t.date <= date(endDate)
    #    and ( t.outDegree = 2 OR t.outDegree = 1 )
    #    and minSum  <= r.value <= minSum * 1.1
    #RETURN t, r.value, vendor.address
    #""")

transaction_t2 = pd.DataFrame()
for i in berlusconi_reviews[berlusconi_reviews['vendor_id'].isin(vendor_wal_filtered['vendor_id'])].itertuples():
    neo_results = []
    neo_results2 = []
    neo_results3 = []

    # time frame in which payout transaction may have occured (start <= payout.date <= end)
    start = str(pd.to_datetime(i.time.date()))
    end = str(pd.to_datetime(i.time.date() + timedelta(days=0)))
    # range of possible inSum values of the payout TA (could also implement the same for outSum, if required)
    minSum = str(round(i.price_btc_reviews_satoshi,0))
    maxSum = str(round(i.price_btc_reviews_satoshi*1.40,0))
    # all addresses within the wallet of a specific vendor
    addresses = str(list(vendor_wal_filtered[vendor_wal_filtered['vendor_id']==i.vendor_id]['Address']))
    
    #neo_query with all possible parameters. Remove whatever you don't want to use
    neo_query = session.run("""
    MATCH (vendor:Address)
        WHERE vendor.address IN """ + addresses + """
    WITH vendor, 
    '""" + start + """' as startDate,
    '""" + end + """' as endDate,
    """+ minSum + """ as minSum, 
    """ + maxSum + """ as maxSum 
    
    MATCH (t:Transaction)-[r:RECEIVES]->(vendor)
        WHERE date(startDate) <= t.date <= date(endDate)
        and ( t.outDegree = 2 OR t.outDegree = 1 )
        and ( t.inDegree = 2 OR t.inDegree = 1 )
        and minSum  <= r.value <= minSum * 1.1
    RETURN t, r.value, vendor.address
    """)
    
    for r in neo_query:
        neo_results.append(r[0])
        neo_results2.append(r[1])
        neo_results3.append(r[2])

    
    df_neo4j = pd.DataFrame(neo_results)
    df_neo4j["vendor"] = i.vendor_id
    df_neo4j["review"] = i.vendor_review_id
    df_neo4j["rvalue"] = pd.Series(neo_results2)
    df_neo4j["vendor_address"] = pd.Series(neo_results3)
    transaction_t2 = pd.concat([transaction_t2,df_neo4j])
    
display(transaction_t2)
#Output explanations:
# vendor := Vendor-ID (MD5-Hash) from sql-review db
# review := review number from sql-review db
# vendor_address := Bitcoin address of Vendor that received the output of the payout transaction

In [ ]:
transaction_t2.to_csv("transaction_t2.csv")

# What is everything below? No comments and I can't remember it..

In [ ]:
'''
results_all_tx=pd.DataFrame()
for i in berlusconi_reviews[berlusconi_reviews['vendor_id'].isin(vendor_wal_filtered['vendor_id'])].itertuples():
    neo_results=[]
    neo_results2=[]

    neo_query = session.run("""
        MATCH (vendor:Address)
            WHERE vendor.address IN """ + addresses + """
        WITH vendor
        MATCH (t:Transaction)-[r:RECEIVES]->(vendor)
        RETURN DISTINCT t
        """)
    for r in neo_query:
        neo_results.append(r[0])
        #neo_results2.append(r[1])
    df_neo4j = pd.DataFrame(neo_results)
    df_neo4j["vendor"] = i.vendor_id
    results_all_tx= pd.concat([results_all_tx,df_neo4j])'''

In [ ]:
#Reviews with too many hits should be disregarded
#transaction_t2.groupby(by="review").count()

#3rd Step

In [4]:
escrow_addresses_v2_withUsername=pd.read_csv('QueryResultsTransaction1EscrowActiveOnBoth0506.csv')
escrow_addresses_v2_withUsername

,Unnamed: 0,inDegree,outDegree,address,vendor,maxDate,minDate,totalMoneyReceived,review,rvalue
0,0,2,2,18Hw3c4WcfRW6cdtfSP8MDsaNHJbN8bpKa,a081033ab42c85ae4101549cd64ff4cb,2017-11-17,2017-10-06,4410000.0,105344.0,4180000.0
1,1,2,2,1BEVjYniwpY4itFtYLhautNpSJqDtPuYre,a081033ab42c85ae4101549cd64ff4cb,2017-10-06,2017-09-27,4571961.0,105344.0,4096761.0
2,2,1,1,15u1M5iN4HAivHSjbgd37RjapYjZwbALG7,a081033ab42c85ae4101549cd64ff4cb,2017-10-06,2017-10-06,4254815.0,105344.0,4254815.0
3,3,22,22,1MR4x3Q4AGUs2PSaXD3HTwV6wpJSky5sgr,a081033ab42c85ae4101549cd64ff4cb,2017-12-27,2017-10-01,64373825.0,105344.0,4218000.0
4,4,22,22,1MR4x3Q4AGUs2PSaXD3HTwV6wpJSky5sgr,a081033ab42c85ae4101549cd64ff4cb,2017-12-27,2017-10-01,64373825.0,105344.0,4218000.0
...,...,...,...,...,...,...,...,...,...,...
71073,6,24,24,3Mepvc6UtQeVqC2XVMbEhcAQUFKeAuC8Lg,5c6b492caa2e2af07032efbaa9a9dd46,2019-07-28,2019-03-01,51634708.0,102688.0,1058973.0
71074,7,4,4,1B4TGoHYBaNFeLGWriyVikWe6GqPe8fsAP,5c6b492caa2e2af07032efbaa9a9dd46,2019-07-29,2019-01-21,52408800.0,102688.0,1047700.0
71075,0,13,13,1AiGyCKxEReTGZfxzkVsAztpeiurnW9vm6,927b6d058152a78d2b3fd80bd12d5778,2019-07-27,2019-07-27,17375144.0,104324.0,444000.0
71076,1,13,13,1AiGyCKxEReTGZfxzkVsAztpeiurnW9vm6,927b6d058152a78d2b3fd80bd12d5778,2019-07-27,2019-07-27,17375144.0,104324.0,444000.0


In [ ]:
escrow_addresses_v3= pd.DataFrame(columns=['escrow', 'vendor', 'relationships'])
for row in escrow_addresses_v2_withUsername[escrow_addresses_v2_withUsername['review'].isin(transaction_t2['review'])].itertuples():
    #vendor=j_addresses[j_addresses['Vendor']==row.username]['Address'].iloc[0]
    t2=transaction_t2[transaction_t2['review']==row.review]['txid'][0]
  
    neo_query = session.run("""
    MATCH p=(escrow:Address {address: '"""+row.address+"""' })-[:RECEIVES|SENDS*1..5]->(t2:Transaction {txid: '"""+ t2+"""'})
    RETURN collect(escrow) as escrow, collect(t2) as t2, collect(last(relationships(p))) as relationships""")
    for r in neo_query:
        escrow=r[0]
        t2=r[1]
        relationships=r[2]
        escrow_addresses_v3.append({'escrow': escrow, 't2':t2, 'relationships':relationships}, ignore_index=True)
    escrow_addresses_v3.to_csv('escrow_addresses_v3.csv')

display(escrow_addresses_v3.head())

escrow_addresses_v3.to_csv('escrow_addresses_v3.csv')

In [5]:
transaction_t2 = pd.read_csv("transaction_t2.csv")

In [ ]:
display(transaction_t2)

In [ ]:
for row in transaction_t2.itertuples():
    print("Starting TA:", row.txid, "\n")
    t2=row.txid
    prior_nodes = []
  
    neo_query = session.run("""
    MATCH path=(a)-[:RECEIVES|SENDS*1..10]->(t2:Transaction)
    WHERE t2.txid = '""" + t2 + """' AND
    ALL(node IN nodes(path) WHERE node.inDegree < 5)
    MATCH (a) WHERE exists(a.address) RETURN a.address""")
    for r in neo_query:
        prior_nodes.append(r[0])
        if r[0] in escrow_addresses_v2_withUsername["address"]:
            print("Match found\ntxid:", t2,", escrow address", r[0], "vendor id", row.vendor)
            
#MATCH (a)-[:RECEIVES|SENDS*1..10]->(t2:Transaction) WHERE t2.txid = '""" + t2 + """' AND a.inDegree < 5

Starting TA: 320cb01774a18f52b2f673699251f922b7b5821d4acd8ae2fab00326cff377d4 

Starting TA: 92b8cd68e73a55cb9208b13099a61df3741bee1f5ef765aa4a3c312564bd37d2 

Starting TA: 70a29688567bd4393b64086011d1762299c350f9e31d4613dd847434592c6819 

Starting TA: a9fc94019bf2cf10806cf3eb7354411a6478c93a6067772cda535b394fe491f1 

Starting TA: 9ab64f5da227f5f6f992bb5ca5b31f89f23d9b3388652b1bfae6193ac1e49ac2 

Starting TA: daa4f71cb9168916394bbe3f999375fd47bdb090878dbd80ead558510fc23fb7 

Starting TA: 04eab619857dcb50bf345bce08ce8266f393102e6a77dcb5bc32316f50f90f4e 



In [ ]:
for row in transaction_t2.itertuples():
    print("Starting TA:", row.txid, "\n")
    t2=row.txid
    prior_nodes = []
  
    neo_query = session.run("""
    MATCH (a:Address) RETURN a.address LIMIT 3""")
    for r in neo_query:
        print("test")
        prior_nodes.append(r[0])
        if r[0] in escrow_addresses_v2_withUsername[escrow_addresses_v2_withUsername["vendor"] == row.vendor]:
            print("Match found\ntxid:", t2,", escrow address", r[0], "vendor id", row.vendor)